In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
import argparse
from tqdm import tqdm
from random import shuffle
from music21 import *
from glob import glob
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout
from keras.utils import np_utils

%run lstm_model.ipynb
%run data_preprocess.ipynb
%run generate_music.ipynb
np.set_printoptions(threshold=1000000)
bach_chorale_path = 'datasets/bach'
parts_index = [0,1,2,3]
#parts_index = [0]
note_type = set()
"""
note_type:
0-127 : note pitch
128: slur-1st-part
129: slur-2nd-part
130: slur-3rd-part
131: slur-4th-part
132: rest
133: start symbol
134: end symbol
"""

def filter_files(files):
    file_list = []
    file_count = 0
    for i, file in enumerate(files, 1):
        s = converter.parse(file)
        if len(s.parts) == len(parts_index):
            print(i, file, len(s.parts))
            file_count += 1
            file_list.append(file)
    return file_list, file_count            


def make_note_type_list():
    note_type_list = []
    for i in note_type:
        note_type_list.append(i)
    return sorted(note_type_list)
# def transform_note_to_index(notes):
#     for time in range(notes.shape[0]):
#         for p_index in range(4):
#             notes[time][p_index] = note_dict[notes[time][p_index]]
#     return notes
def index_note_transform(inputs, option, pn):
    transformed_inputs = []
    if option == "to_note":
        note_dict = dict((index, note) for index, note in enumerate(note_type))
    elif option == "to_index":
        note_dict = dict((note, index) for index, note in enumerate(note_type))
        
    for p_index in range(pn):
        transformed_part = []
        for n in range(inputs.shape[1]):
            for keys in note_dict:
                if option == "to_note":
                    if inputs[p_index][n] == keys:
                        transformed_part.append(note_dict[keys])
                elif option == "to_index":
                    if frozenset(inputs[p_index][n]) == keys:
                        transformed_part.append(note_dict[keys])
        transformed_inputs.append(transformed_part)
    return np.array(transformed_inputs)
def notes_to_piano_roll(dataset_notes):
#     start_symbol = np.full((16, 1),133)
#     end_symbol = np.full((16, 1),134)
    note_type.add(frozenset([133]))
    note_type.add(frozenset([134]))
    for chorale_index in range(len(dataset_notes)):
        chorale = np.array(dataset_notes[chorale_index])
#         for i in range(len(parts_index)):
#             piano_roll_notes.append(np.concatenate((start_symbol, chorale[i], end_symbol)))
        chorale = index_note_transform(chorale, "to_index", len(parts_index))
        piano_roll_notes = np.transpose(chorale)
        #piano_roll_notes = index_note_transform(transposed_chorale, "to_index", 4)
        yield piano_roll_notes
def notes_to_onehot(notes):
    note_dict = note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = []
        for note in n:
            a.append(np.array(note == np.arange(0, len(note_dict)), dtype=np.int32))
        onehot.append(a)
    return np.array(onehot)
def part_notes_to_onehot(notes):
    note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = np.zeros(len(note_dict,),dtype=np.int32)
        for note in n:
            a[note] = 1
        onehot.append(a)
    return np.array(onehot)

def split_data(chorale_list):
    X_train = chorale_list[0]
    X_test = chorale_list[int(len(chorale_list) * 0.8)]
    notes = chorale_list[0]
    
    for i, n in enumerate(chorale_list):
        if i != 0:
            notes = np.concatenate((notes, n))
            if i < int(len(chorale_list) * 0.8):
                X_train = np.concatenate((X_train, n))
            elif i > int(len(chorale_list) * 0.8):
                X_test = np.concatenate((X_test, n))
    print(X_train.shape, X_test.shape, notes.shape)
    
    X_train_onehot_notes = part_notes_to_onehot(X_train)
    X_test_onehot_notes = part_notes_to_onehot(X_test)
    Y_train_onehot_notes = notes_to_onehot(X_train)
    Y_test_onehot_notes = notes_to_onehot(X_test)
    onehot_notes = part_notes_to_onehot(notes)
    
    return X_train_onehot_notes, X_test_onehot_notes, Y_train_onehot_notes, Y_test_onehot_notes, onehot_notes
def load_files():
    if os.path.exists(bach_chorale_path):
#         file_list = filter_files(glob(bach_chorale_path))
        file_list, num_file = filter_files(glob(bach_chorale_path + '/*.mxl') +
                                 glob(bach_chorale_path + '/*.mid'))
        return file_list, num_file
    else:
        print("The file path is wrong!!")
    
def main():
#     parser = argparse.ArgumentParser(description='**********************')
#     parser.add_argument('-t', '--train')
#     parser.add_argument('-g', '--generate', help='generate music with trained model')
#     args = parser.parse_args()
    train = True
    file_list, num_file = load_files()
    print(num_file)
    dataset_notes = make_dataset(file_list)
    shuffle(dataset_notes)
    chorale_list = [notes for chorale_index, notes in enumerate(notes_to_piano_roll(dataset_notes))]
    
#     X_train = chorale_list[0]
#     X_test = chorale_list[int(len(chorale_list) * 0.8)]
#     notes = chorale_list[0]
        
#     for i, n in enumerate(chorale_list):
#         if i != 0:
#             notes = np.concatenate((notes, n))
#             if i < int(len(chorale_list) * 0.8):
#                 X_train = np.concatenate((X_train, n))
#             elif i > int(len(chorale_list) * 0.8):
#                 X_test = np.concatenate((X_test, n))
    
    
#     X_train_onehot_notes = part_notes_to_onehot(X_train)
#     X_test_onehot_notes = part_notes_to_onehot(X_test)
#     Y_train_onehot_notes = notes_to_onehot(X_train)
#     Y_test_onehot_notes = notes_to_onehot(X_test)
    X_train_onehot_notes, X_test_onehot_notes, Y_train_onehot_notes, Y_test_onehot_notes, onehot_notes = split_data(chorale_list)
    epochs = 15
    if train:
        for i in range(len(parts_index)): 
            lstm_model = LSTM_Model(X=X_train_onehot_notes,Y=Y_train_onehot_notes,
                                    num_units=128, epochs=epochs, output_dim=len(note_type), name="bach"+str(i))
            gen = ((X,Y) for (X, Y) in lstm_model.generator(X_train_onehot_notes, Y_train_onehot_notes, i))
            test_gen = ((X,Y) for (X,Y) in lstm_model.generator(X_test_onehot_notes, Y_test_onehot_notes, i))
            lstm_model.create_model()
            lstm_model.train_model(gen, test_gen)
    print(onehot_notes.shape)
    score = generate_notes(onehot_notes)
                
    output_file = str(epochs) + '-epochs_' + str(num_file) + '-samples.mid'
    mf = midi.translate.music21ObjectToMidiFile(score)
    mf.open(output_file, 'wb')
    mf.write()
    mf.close()
    print("File " + output_file + " written")
if __name__ == "__main__":
    main()

2 datasets/bach\bwv10.7.mxl 4
3 datasets/bach\bwv101.7.mxl 4
4 datasets/bach\bwv102.7.mxl 4
5 datasets/bach\bwv103.6.mxl 4
6 datasets/bach\bwv104.6.mxl 4
7 datasets/bach\bwv108.6.mxl 4
8 datasets/bach\bwv11.6.mxl 4
9 datasets/bach\bwv110.7.mxl 4
10 datasets/bach\bwv111.6.mxl 4
12 datasets/bach\bwv112.5.mxl 4
13 datasets/bach\bwv113.8.mxl 4
14 datasets/bach\bwv114.7.mxl 4
15 datasets/bach\bwv115.6.mxl 4
16 datasets/bach\bwv116.6.mxl 4
17 datasets/bach\bwv117.4.mxl 4
18 datasets/bach\bwv119.9.mxl 4
20 datasets/bach\bwv120.6.mxl 4
22 datasets/bach\bwv121.6.mxl 4
23 datasets/bach\bwv122.6.mxl 4
24 datasets/bach\bwv123.6.mxl 4
26 datasets/bach\bwv125.6.mxl 4
27 datasets/bach\bwv126.6.mxl 4
28 datasets/bach\bwv127.5.mxl 4
30 datasets/bach\bwv13.6.mxl 4
32 datasets/bach\bwv135.6.mxl 4
35 datasets/bach\bwv139.6.mxl 4
36 datasets/bach\bwv14.5.mxl 4
37 datasets/bach\bwv140.7.mxl 4
38 datasets/bach\bwv144.3.mxl 4
39 datasets/bach\bwv144.6.mxl 4
40 datasets/bach\bwv145-a.mxl 4
41 datasets/bach\bwv

291 datasets/bach\bwv39.7.mxl 4
292 datasets/bach\bwv390.mxl 4
293 datasets/bach\bwv391.mxl 4
294 datasets/bach\bwv392.mxl 4
295 datasets/bach\bwv393.mxl 4
296 datasets/bach\bwv394.mxl 4
297 datasets/bach\bwv395.mxl 4
298 datasets/bach\bwv396.mxl 4
299 datasets/bach\bwv397.mxl 4
300 datasets/bach\bwv398.mxl 4
301 datasets/bach\bwv399.mxl 4
302 datasets/bach\bwv4.8.mxl 4
303 datasets/bach\bwv40.3.mxl 4
304 datasets/bach\bwv40.6.mxl 4
305 datasets/bach\bwv40.8.mxl 4
306 datasets/bach\bwv400.mxl 4
307 datasets/bach\bwv401.mxl 4
308 datasets/bach\bwv402.mxl 4
309 datasets/bach\bwv403.mxl 4
310 datasets/bach\bwv404.mxl 4
311 datasets/bach\bwv405.mxl 4
312 datasets/bach\bwv406.mxl 4
313 datasets/bach\bwv407.mxl 4
314 datasets/bach\bwv408.mxl 4
316 datasets/bach\bwv410.mxl 4
317 datasets/bach\bwv411.mxl 4
318 datasets/bach\bwv412.mxl 4
319 datasets/bach\bwv413.mxl 4
320 datasets/bach\bwv414.mxl 4
321 datasets/bach\bwv415.mxl 4
322 datasets/bach\bwv416.mxl 4
323 datasets/bach\bwv417.mxl 4
324 

100%|████████████████████████████████████████████████████████████████████████████████| 352/352 [00:43<00:00, 16.99it/s]                                                                                                                       


(80000, 4) (19986, 4) (99986, 4)
create model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 16, 128)           93696     
_________________________________________________________________
dropout_27 (Dropout)         (None, 16, 128)           0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 16, 128)           131584    
_________________________________________________________________
dropout_28 (Dropout)         (None, 16, 128)           0         
_________________________________________________________________
dense_14 (Dense)             (None, 16, 54)            6966      
Total params: 232,246
Trainable params: 232,246
Non-trainable params: 0
_________________________________________________________________
train model...
Epoch 1/15
256/256 [==============================] - ETA: 2427s - loss:

256/256 [==============================] - ETA: 31s - loss: 0.73 - ETA: 32s - loss: 0.62 - ETA: 32s - loss: 0.56 - ETA: 33s - loss: 0.51 - ETA: 33s - loss: 0.50 - ETA: 33s - loss: 0.47 - ETA: 33s - loss: 0.50 - ETA: 33s - loss: 0.49 - ETA: 33s - loss: 0.49 - ETA: 32s - loss: 0.57 - ETA: 32s - loss: 0.72 - ETA: 32s - loss: 0.85 - ETA: 32s - loss: 0.86 - ETA: 32s - loss: 0.84 - ETA: 32s - loss: 0.84 - ETA: 32s - loss: 0.84 - ETA: 32s - loss: 0.84 - ETA: 32s - loss: 0.85 - ETA: 32s - loss: 0.87 - ETA: 31s - loss: 0.85 - ETA: 31s - loss: 0.84 - ETA: 31s - loss: 0.84 - ETA: 31s - loss: 0.84 - ETA: 31s - loss: 0.83 - ETA: 31s - loss: 0.82 - ETA: 31s - loss: 0.84 - ETA: 30s - loss: 0.84 - ETA: 30s - loss: 0.89 - ETA: 30s - loss: 0.97 - ETA: 30s - loss: 0.97 - ETA: 30s - loss: 0.96 - ETA: 30s - loss: 0.95 - ETA: 30s - loss: 0.95 - ETA: 30s - loss: 0.95 - ETA: 29s - loss: 0.93 - ETA: 29s - loss: 0.92 - ETA: 29s - loss: 0.92 - ETA: 29s - loss: 0.91 - ETA: 29s - loss: 0.91 - ETA: 28s - loss: 0.90

...

256/256 [==============================] - ETA: 40s - loss: 0.00 - ETA: 39s - loss: 0.02 - ETA: 38s - loss: 0.01 - ETA: 39s - loss: 0.04 - ETA: 38s - loss: 0.03 - ETA: 37s - loss: 0.05 - ETA: 36s - loss: 0.04 - ETA: 37s - loss: 0.04 - ETA: 37s - loss: 0.04 - ETA: 37s - loss: 0.04 - ETA: 37s - loss: 0.05 - ETA: 37s - loss: 0.05 - ETA: 38s - loss: 0.05 - ETA: 37s - loss: 0.10 - ETA: 37s - loss: 0.12 - ETA: 37s - loss: 0.12 - ETA: 38s - loss: 0.11 - ETA: 37s - loss: 0.11 - ETA: 37s - loss: 0.10 - ETA: 37s - loss: 0.10 - ETA: 37s - loss: 0.09 - ETA: 36s - loss: 0.09 - ETA: 36s - loss: 0.08 - ETA: 36s - loss: 0.10 - ETA: 36s - loss: 0.09 - ETA: 36s - loss: 0.09 - ETA: 36s - loss: 0.09 - ETA: 36s - loss: 0.09 - ETA: 36s - loss: 0.09 - ETA: 35s - loss: 0.08 - ETA: 35s - loss: 0.08 - ETA: 35s - loss: 0.08 - ETA: 35s - loss: 0.08 - ETA: 35s - loss: 0.08 - ETA: 34s - loss: 0.08 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07

256/256 [==============================] - ETA: 53s - loss: 0.01 - ETA: 58s - loss: 0.02 - ETA: 58s - loss: 0.09 - ETA: 54s - loss: 0.09 - ETA: 53s - loss: 0.12 - ETA: 52s - loss: 0.11 - ETA: 51s - loss: 0.10 - ETA: 50s - loss: 0.09 - ETA: 49s - loss: 0.08 - ETA: 49s - loss: 0.07 - ETA: 48s - loss: 0.09 - ETA: 47s - loss: 0.08 - ETA: 46s - loss: 0.09 - ETA: 46s - loss: 0.08 - ETA: 47s - loss: 0.08 - ETA: 47s - loss: 0.07 - ETA: 47s - loss: 0.07 - ETA: 47s - loss: 0.07 - ETA: 47s - loss: 0.06 - ETA: 47s - loss: 0.06 - ETA: 46s - loss: 0.06 - ETA: 45s - loss: 0.06 - ETA: 44s - loss: 0.06 - ETA: 43s - loss: 0.07 - ETA: 42s - loss: 0.08 - ETA: 42s - loss: 0.08 - ETA: 42s - loss: 0.08 - ETA: 42s - loss: 0.09 - ETA: 43s - loss: 0.09 - ETA: 43s - loss: 0.09 - ETA: 43s - loss: 0.09 - ETA: 43s - loss: 0.09 - ETA: 43s - loss: 0.08 - ETA: 42s - loss: 0.09 - ETA: 42s - loss: 0.09 - ETA: 42s - loss: 0.09 - ETA: 42s - loss: 0.08 - ETA: 41s - loss: 0.08 - ETA: 41s - loss: 0.08 - ETA: 41s - loss: 0.08

256/256 [==============================] - ETA: 38s - loss: 0.01 - ETA: 36s - loss: 0.01 - ETA: 38s - loss: 0.01 - ETA: 39s - loss: 0.01 - ETA: 39s - loss: 0.01 - ETA: 39s - loss: 0.01 - ETA: 38s - loss: 0.01 - ETA: 38s - loss: 0.01 - ETA: 40s - loss: 0.01 - ETA: 40s - loss: 0.03 - ETA: 39s - loss: 0.12 - ETA: 38s - loss: 0.14 - ETA: 38s - loss: 0.13 - ETA: 37s - loss: 0.13 - ETA: 37s - loss: 0.12 - ETA: 36s - loss: 0.12 - ETA: 36s - loss: 0.11 - ETA: 35s - loss: 0.11 - ETA: 35s - loss: 0.11 - ETA: 35s - loss: 0.10 - ETA: 35s - loss: 0.10 - ETA: 35s - loss: 0.09 - ETA: 35s - loss: 0.09 - ETA: 35s - loss: 0.08 - ETA: 35s - loss: 0.09 - ETA: 35s - loss: 0.09 - ETA: 35s - loss: 0.09 - ETA: 34s - loss: 0.08 - ETA: 34s - loss: 0.08 - ETA: 34s - loss: 0.08 - ETA: 35s - loss: 0.08 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07 - ETA: 34s - loss: 0.07 - ETA: 33s - loss: 0.06 - ETA: 33s - loss: 0.06 - ETA: 33s - loss: 0.06

256/256 [==============================] - ETA: 29s - loss: 0.21 - ETA: 28s - loss: 0.11 - ETA: 28s - loss: 0.07 - ETA: 28s - loss: 0.05 - ETA: 27s - loss: 0.04 - ETA: 28s - loss: 0.04 - ETA: 27s - loss: 0.05 - ETA: 27s - loss: 0.05 - ETA: 29s - loss: 0.04 - ETA: 31s - loss: 0.04 - ETA: 30s - loss: 0.04 - ETA: 30s - loss: 0.04 - ETA: 30s - loss: 0.04 - ETA: 30s - loss: 0.04 - ETA: 30s - loss: 0.04 - ETA: 29s - loss: 0.05 - ETA: 29s - loss: 0.06 - ETA: 29s - loss: 0.06 - ETA: 28s - loss: 0.06 - ETA: 28s - loss: 0.06 - ETA: 28s - loss: 0.06 - ETA: 28s - loss: 0.05 - ETA: 27s - loss: 0.05 - ETA: 27s - loss: 0.05 - ETA: 28s - loss: 0.05 - ETA: 28s - loss: 0.05 - ETA: 28s - loss: 0.04 - ETA: 28s - loss: 0.04 - ETA: 28s - loss: 0.04 - ETA: 27s - loss: 0.04 - ETA: 27s - loss: 0.04 - ETA: 27s - loss: 0.04 - ETA: 27s - loss: 0.04 - ETA: 28s - loss: 0.04 - ETA: 27s - loss: 0.04 - ETA: 27s - loss: 0.04 - ETA: 27s - loss: 0.04 - ETA: 27s - loss: 0.04 - ETA: 27s - loss: 0.03 - ETA: 27s - loss: 0.03

  frozenset({131}) frozenset({131}) frozenset({41}) frozenset({131})]]
File 15-epochs_352-samples.mid written


In [ ]:
n = [0,1,2,3,4,5,6,7,8]
n = np.array(n)
g = np.zeros((len(n),),dtype=np.int32)
print(g)
a = [0,5,7]
for i in a:
    print(np.array(i == np.arange(0, len(n) )))

In [86]:
def inner_pro(x1,x2,y1,y2):
    return (x1*y1)+(x2*y2)
x = [1.4, 1.6]
A1 = [1.5, 2.0, 1.6, 1.2, 1.5]
A2 = [1.7, 1.9, 1.8, 1.5, 1.0]
ans = []
for i in range(5):
    ans.append(inner_pro(x[0],x[1],A1[i],A2[i]) / (inner_pro(x[0],x[1],x[0],x[1]) * inner_pro(A1[i],A2[i],A1[i],A2[i])))
ans

[0.2074653076684687,
 0.16978126126545184,
 0.19530057979859625,
 0.2446219152457011,
 0.2518720217835262]

In [68]:
f = frozenset([1,2,3,4,5])
for i in f:
    print(i)

5
